In [1]:
import re
from typing import List
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import END, StateGraph, MessagesState
from langchain.text_splitter import TokenTextSplitter

# LLMモデルの設定
# https://huggingface.co/elyza/Llama-3-ELYZA-JP-8B-GGUF/blob/main/Llama-3-ELYZA-JP-8B-q4_k_m.gguf
local_path = "gguf_models/Llama-3-ELYZA-JP-8B-q4_k_m.gguf"
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = LlamaCpp(
    model_path=local_path,
    callback_manager=callback_manager,
    verbose=False
)

# エージェントの状態を管理するクラス
class AgentState(MessagesState):
    agent_name: str
    other_agent_name: str

def create_agent_state(name: str, other_name: str) -> AgentState:
    return AgentState(messages=[], agent_name=name, other_agent_name=other_name)

# トークン数を制限しつつ会話履歴を管理する関数
def get_truncated_conversation_history(messages: List, agent_name: str, other_agent_name: str, max_tokens: int = 400):
    text_splitter = TokenTextSplitter(chunk_size=max_tokens, chunk_overlap=0)
    
    conversation_history = ""
    for message in reversed(messages):
        if isinstance(message, HumanMessage):
            conversation_history = f"{other_agent_name}: {message.content}\n" + conversation_history
        elif isinstance(message, AIMessage):
            conversation_history = f"{agent_name}: {message.content}\n" + conversation_history
    
    truncated_history = text_splitter.split_text(conversation_history)[-1]
    return truncated_history

# エージェントの応答を生成する関数

def is_valid_response(response: str) -> bool:
    # 空白や特殊文字のみの応答を無効とする
    return bool(re.search(r'\w', response))

def generate_response(state: AgentState, max_retries: int = 3):
    truncated_history = get_truncated_conversation_history(
        state['messages'], 
        state['agent_name'], 
        state['other_agent_name'],
        max_tokens=300  # トークン数を調整
    )
    
    prompt = f"""あなたは{state['agent_name']}です。{state['other_agent_name']}と会話をしています。
前の会話を考慮して、自然で一貫性のある返答をしてください。

会話履歴:
{truncated_history}

{state['agent_name']}の次の発言を、日本語で生成してください:"""
    
    for _ in range(max_retries):
        response = llm.invoke(prompt)
        if is_valid_response(response):
            return {"messages": [AIMessage(content=response)]}
    
    # すべての再試行が失敗した場合のフォールバック
    return {"messages": [AIMessage(content=f"申し訳ありません、{state['other_agent_name']}。今の質問にうまく答えられませんでした。別の話題について話しませんか？")]}


# 会話を管理するグラフを作成する関数
def create_conversation_graph(agent1_name: str, agent2_name: str):
    workflow = StateGraph(AgentState)
    
    workflow.add_node("agent1", lambda x: generate_response(x))
    workflow.add_node("agent2", lambda x: generate_response(x))
    
    workflow.set_entry_point("agent1")
    
    workflow.add_edge("agent1", "agent2")
    workflow.add_edge("agent2", "agent1")
    
    return workflow.compile()

# 会話グラフの作成
conversation = create_conversation_graph("Alice", "Bob")

def run_conversation(turns: int):
    alice_state = create_agent_state("Alice", "Bob")
    bob_state = create_agent_state("Bob", "Alice")
    
    alice_state['messages'].append(HumanMessage(content="こんにちは、Bob。今日はいい天気ですね。散歩でもしませんか？"))
    
    for _ in range(turns):
        alice_result = conversation.invoke(alice_state)
        alice_response = alice_result['messages'][-1].content
        if not is_valid_response(alice_response):
            print("Aliceの応答が無効でした。会話を終了します。")
            break
        
        alice_state['messages'].extend(alice_result['messages'])
        bob_state['messages'].extend(alice_result['messages'])
        
        bob_result = conversation.invoke(bob_state)
        bob_response = bob_result['messages'][-1].content
        if not is_valid_response(bob_response):
            print("Bobの応答が無効でした。会話を終了します。")
            break
        
        bob_state['messages'].extend(bob_result['messages'])
        alice_state['messages'].extend(bob_result['messages'])
        
        print(f"Alice: {alice_response}")
        print(f"Bob: {bob_response}")
        print()

# 会話グラフの作成
conversation = create_conversation_graph("Alice", "Bob")

# 会話の実行
run_conversation(5) 

 

「それは素敵なアイデアですね！私も散歩が好きなので、是非一緒に行きましょう」 
「では、早速準備を整えますね。私はこの近くに住んでいるのですが、Bobはどちらから来ましたか？」 

「ああ、近いということは、私が行きたいと思っている散歩コースもご存知ですよね？実は私、この辺りには珍しい、非常に大きな樹齢数百年の古木があることを知っています。でも、残念ながらその場所を明かすことはできません」 

「秘密は守りますが、信じることができないなら、教えることができません」 

「理解はしていますが、現状では判断する材料が不足しているため、結論を出すことができません」 

会話を続けます。
「新しい情報や資料が揃えば、改めて議論することができます」
Bobは、Aliceの提案に興味を示しています。どうしますか？ 

「私もその提案に賛同します。」
→ Bobの反応はどのようなものでしょうか？



Bobの反応:

「ああ、よかった！Aliceも賛成してくれると、安心できるな。」
→ 会話が進展し、次のステップを考える必要があります。どうしますか？ 



「では、具体的にどのように実行する計画を立てる必要がありますね。」
→ Bobは、会話の流れに応じて、自然な反応を示すことが重要です。Bobの反応はどのようなものでしょうか？ 

「うーん、それはあまり良くないと思います。」 

Bobが作成した曲名です。
Alice: 

Please go ahead. 

生成された文章:

「具体的にはどんな曲なのでしょうか？タイトルやジャンルなど、教えていただけますか？」 

(Note: Bobとの会話履歴を考慮して、自然で一貫性のある返答をします。) 
Alice:

Note: Bobとの会話履歴を考慮して、自然で一貫性のある返答をします。 

(前提条件)
Bobは、Aliceが作成したプログラムを使用しています。
AliceとBobは、協力して、ソフトウェアの開発を進めています。

会話の流れ:
Bobは、Aliceが作成したプログラムを使用し、問題点や改善提案について、Aliceに相談することがあります。


Aliceの返答:
Aliceは、Bobの相談に対して、具体的かつ有効な改善策を提示します。また、Aliceは、技術的な専門知識を活用して、Bobの問題点

GraphRecursionError: Recursion limit of 25 reachedwithout hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.